In [1]:
# imports
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama 
from IPython.display import Image, display
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langgraph.graph import StateGraph, START, END


In [2]:
import os, getpass
from dotenv import load_dotenv
load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [3]:
import google.generativeai as genai
from PIL import Image

model_gemini = genai.GenerativeModel('models/gemini-2.0-flash')

image = r"E:\Deep_Learning\Skin diseases\data_bite\train\fleas\fleas_3.jpg"
image = Image.open(image)

prompt = '''You are a medical image classification assistant.
Given an image of a skin condition, classify it into one of the following categories:

1. Mosquito Bites
2. Bedbug Bites
3. Tick Bites
4. Flea Bites
5. Chigger Bites
6. Spider Bites
7. Ant Bites
8. Bee Stings

Please analyze the image and respond ONLY in the following dictionary format:

{
  "predicted_class": "<class name from above>",
  "prediction_confidences": {
    "Mosquito Bites": <probability between 0 and 1>,
    "Bedbug Bites": <probability between 0 and 1>,
    "Tick Bites": <probability between 0 and 1>,
    "Flea Bites": <probability between 0 and 1>,
    "Chigger Bites": <probability between 0 and 1>,
    "Spider Bites": <probability between 0 and 1>,
    "Ant Bites": <probability between 0 and 1>,
    "Bee Stings": <probability between 0 and 1>
  }
}

Where:
- "predicted_class" is the class with the highest probability.
- "prediction_confidences" is a dictionary with the probability for each class (all values between 0 and 1, and should sum to 1).

If you are unsure, distribute the probabilities accordingly.
'''

response = model_gemini.generate_content([prompt, image]  )
response.text

'```json\n{\n  "predicted_class": "Mosquito Bites",\n  "prediction_confidences": {\n    "Mosquito Bites": 0.5,\n    "Bedbug Bites": 0.2,\n    "Tick Bites": 0.05,\n    "Flea Bites": 0.1,\n    "Chigger Bites": 0.05,\n    "Spider Bites": 0.05,\n    "Ant Bites": 0.03,\n    "Bee Stings": 0.02\n  }\n}\n```'

# Architecture 

![Medical RAG](images/Medical_RAG_Application.png)

# Graph

1. Add Image processing, as a Node with will return probability of belongig to a given class with and class ✔️
2. RAG (retrieval)                                                                                          ✔️
4. LLM response                                                                                             ✔️
5. Conversation Summarization                                                                               ✔️
6. Smart query routing                                                                                      ✔️
7. Build User Interface with streamlit



# LLMs

In [4]:
model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")
embedings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
#model = ChatOllama(model="deepseek-r1:7b", temperature=0.1)
model_for_query_route = ChatOllama(model = "llama3.2:latest")
#embedings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")


# Prompts

In [5]:
template = '''
# Role
You are a medical AI assistant focused on Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system.
Your primary goal is to provide precise answers based on the given context, image analysis, chat history, and conversation summary.

# Instruction
Provide a concise, logical answer by organizing the selected content into coherent paragraphs with a natural flow. 
Avoid merely listing information. Include key numerical values, technical terms, jargon, and names. 
DO NOT use any outside knowledge or information that is not in the given material.

# Constraint
- Review the provided context, image analysis, chat history, and conversation summary thoroughly and extract key details related to the question.
- Craft a precise answer based on the relevant information.
- Keep the answer concise but logical/natural/in-depth.
- If the retrieved context does not contain relevant information or no context is available, respond with: 'I can't find the answer to that question in the context.'

**Source** 
- Cite the source of the information as a file name with a page number or URL, omitting the source if it cannot be identified.
- (list more if there are multiple sources)

# Sources
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
[1] Link or Document name  
[2] Link or Document name

- Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/  
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/

# Conversation Summary
<summary>
{summary}
</summary>

# Question
<question>
{question}
</question>

# Image Analysis
<image_analysis>
The image has been classified as: {predicted_class}
Prediction confidence: {confidence:.2%}
</image_analysis>

# Context
<retrieved context>
{context}
</retrieved context>

# Answer
'''

template_for_no_image = '''
# Role
You are a medical AI assistant focused on Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system.
Your primary goal is to provide precise answers based on the given context, chat history, and conversation summary.

# Instruction
Provide a concise, logical answer by organizing the selected content into coherent paragraphs with a natural flow. 
Avoid merely listing information. Include key numerical values, technical terms, jargon, and names. 
DO NOT use any outside knowledge or information that is not in the given material.

# Constraint
- Review the provided context, chat history, and conversation summary thoroughly and extract key details related to the question.
- Craft a precise answer based on the relevant information.
- Keep the answer concise but logical/natural/in-depth.
- If the retrieved context does not contain relevant information or no context is available, respond with: 'I can't find the answer to that question in the context.'

**Source** 
- Cite the source of the information as a file name with a page number or URL, omitting the source if it cannot be identified.
- (list more if there are multiple sources)

# Sources
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
[1] Link or Document name  
[2] Link or Document name

- Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/  
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/

# Conversation Summary
<summary>
{summary}
</summary>

# Question
<question>
{question}
</question>

# Context
<retrieved context>
{context}
</retrieved context>

# Answer

'''

route_question_template = '''
# Role
You are an expert system that routes user health-related questions to either a vectorstore or a web search engine.

# Vectorstore Contents
The retrive contains documents related to:
- Wounds and cuts: how to clean, treat, and care for them
- Insect bites and stings: symptoms, treatments, and prevention
- Skin injuries caused by physical trauma or bites

Use the vectorstore if the user's question clearly falls into one of these categories.

# Routing Rules
- If the question concerns wound care, insect bites, stings, or minor skin injuries — use the **vectorstore**.
- For all other medical topics (e.g. internal diseases, medication side effects, diagnostics, etc.) or unrelated queries — use **web search**.

Always make a clear and accurate decision between the two options: `retrive` or `websearch`.
'''

grade_documents_template = """
You are an expert AI system for document evaluation.

Your task is to assess whether the provided documents contain information that directly helps to answer the given question.

Return ONLY:
- True — if the documents are relevant to the question.
- False — if the documents are not relevant to the question.

Guidelines:
- "Relevant" means the documents either directly answer or clearly help to answer the question.
- If the documents are off-topic, missing key details, or only slightly related, they are considered NOT relevant.
- Do not infer or guess beyond the given content. Only judge based on the text provided.

Respond ONLY with True or False, without any explanation.
"""

check_hallucination_template = """
You are an expert fact-checker.

Your task is to verify if the provided LLM-generated answer is fully grounded in the provided documents (facts).

Return ONLY:
- True — if the answer is directly supported by the documents.
- False — if the answer contains information not found or contradicted by the documents.

Guidelines:
- Only accept answers that are clearly based on the provided facts.
- If the answer includes invented, assumed, or unrelated information — mark as False.
- Do not infer missing information or "fill gaps". Only check based on the given facts.

Respond only with True or False, without any explanation."""

answer_question_template = """
You are a grader assessing whether an AI-generated answer successfully addresses the user's question.

Return ONLY:
- True — if the answer correctly and clearly responds to the user's question.
- False — if the answer is irrelevant, incomplete, off-topic, or wrong.

Guidelines:
- An answer is acceptable if it is correct, clear, and meaningfully answers the question.
- If the answer is too vague, partially correct, or off-topic — mark as False.
- Focus only on the relation between the question and the provided answer.

Respond only with True or False, without any additional comments."""

# Create vector database

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
import bs4

web_paths = ['https://medlineplus.gov/ency/article/000033.html',
             'https://pmc.ncbi.nlm.nih.gov/articles/PMC10503338/',
             'https://www.ncbi.nlm.nih.gov/books/NBK537235/',
             'https://medicaljournalssweden.se/actadv/article/view/11592/19144',
             'https://emedicine.medscape.com/article/769067-overview?form=fpf',
             'https://www.mayoclinic.org/first-aid/first-aid-insect-bites/basics/art-20056593',
             'https://www.medicalnewstoday.com/articles/174229#reactions',
             'https://www.aafp.org/pubs/afp/issues/2022/0800/arthropod-bites-stings.html',
             'https://wwwnc.cdc.gov/travel/page/avoid-bug-bites',
             'https://www.webmd.com/first-aid/ss/slideshow-caring-for-wounds',
             'https://www.mayoclinic.org/first-aid/first-aid-cuts/basics/art-20056711',
             'https://www.betterhealth.vic.gov.au/health/conditionsandtreatments/wounds-how-to-care-for-them']

loader = WebBaseLoader(
    web_paths=web_paths,
)

documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                         chunk_overlap=400,
                                         add_start_index=True)

all_splits = splitter.split_documents(documents)

print(f"Split website into {len(all_splits)} sub-documents.")



vectorstores = FAISS.from_documents(documents=all_splits,
                                    embedding=embedings)

retriver = vectorstores.as_retriever(k=2)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Split website into 386 sub-documents.


In [7]:
def retrive(state):
    """
    Retrieve relevant documents based on image classification and user question.

    Args:
        state (dict): The current graph state, including 'image_metadata' and 'question'.

    Returns:
        dict: New state with a key 'retrived_docs' containing the retrieved documents.
    """
    
    print("---RETRIVE---")
    question = state.get("question")

    if state.get("image") is not None:
        prediction = state.get("image_metadata", {})
        cls = prediction.get("predicted_class")
        conf = prediction.get("prediction_confidences")

        probability = conf.get(cls , 0)
    
        query = f"This question relates to a medical image classified as: {cls} with probability {probability}"

        print(f"Retrieval query: {query}")
    
        documents = retriver.invoke(query)

    else: 
        print(f"Retrieval query:{question}")
        documents = retriver.invoke(question)
        
    return {
        "retrived_docs" : [documents]
    }
    

# Image processing

In [8]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


model_path = r"E:\Deep_Learning\Skin diseases\best_model.h5"
model_cv = load_model(model_path)

BATCH_SIZE = 64
IMAGE_SIZE = 150
channels = 3
classes = ['Bite', 'Healthy Skin', 'Wound']

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    
    if img is None:
        raise ValueError(f"Image not loaded correctly from path: {image_path}")
        
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = img.reshape(1, IMAGE_SIZE, IMAGE_SIZE, channels)
    img = img / 255.0
    
    return img


def analyze_image_node(state):
    """
    Analyze the image and generate predictions for classification.

    Args:
        state (dict): The current graph state containing the 'image' path.

    Returns:
        dict: New state with a key 'image_metadata' including predicted class and confidence scores.
    """
    
    print("---IMAGE ANALYSIS---")
    
    image_path = state.get("image")

    if not image_path:
        print("No image provided to input")
        return {}

    # Preprocess + prediction
    img = preprocess_image(image_path)
    prediction = model_cv.predict(img)
    
    confidences = {cls: float(conf) for cls, conf in zip(classes, prediction[0])}
    predicted_class = max(confidences, key=confidences.get)

    return {
        "image_metadata": {
            "predicted_class": predicted_class,
            "prediction_confidences": confidences
        }
    }



# Generate Response

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def generate_response(state):
    """
    Generate a final answer based on the retrieved documents, classification result, and user question.

    Args:
        state (dict): The current graph state containing 'image_metadata', 'retrived_docs', and 'question'.

    Returns:
        dict: New state with a key 'answer' containing the generated response.
    """
    print("---GENERATION OF RESPONSE---")

    question = state.get("question")
    retrived_docs = state.get("retrived_docs")
    summary = state.get("summary", "")
    cls = None
    conf = None

    if state.get("image"):
        prediction = state.get("image_metadata", {})
        cls = prediction.get("predicted_class")
        conf = prediction.get("prediction_confidences", {}).get(cls, 0)

    prompt_template = template if state.get("image") else template_for_no_image

    prompt = ChatPromptTemplate.from_template(prompt_template)
    
    model_response = model.invoke(
        prompt.format(
            question=question,
            context=retrived_docs,
            predicted_class=cls,
            confidence=conf,
            summary=summary or "There is no chat history yet"
        )
    )
    token_count_usage = model_response.usage_metadata
    token_count = state.get("token_count", [])
    if not isinstance(token_count, list):
        token_count = []  

    token_count.append(token_count_usage)
    return {
        "answer": [model_response],
        "token_usage" : token_count
    }

In [10]:
response = model.invoke("Hi what i can do for you?")
tokens = response.usage_metadata
print(f"Token count: {tokens}")
print(response)

Token count: None
content='I\'m here to help you with a wide range of tasks! Here are some examples of things I can do:\n\n*   **Answer questions:** I can provide information on a vast array of topics, from science and history to current events and pop culture.\n*   **Generate text:** I can write stories, poems, articles, scripts, and other creative content.\n*   **Summarize information:** I can condense lengthy articles or documents into shorter, more manageable summaries.\n*   **Translate languages:** I can translate text between multiple languages.\n*   **Write different kinds of creative content:** I can generate different creative text formats, like poems, code, scripts, musical pieces, email, letters, etc. I will try my best to fulfill all your requirements.\n*   **Brainstorm ideas:** I can help you come up with ideas for projects, events, or solutions to problems.\n*   **Provide definitions:** I can define words and concepts.\n*   **Offer suggestions:** I can suggest books, movi

In [11]:
def test_generate_response():
    """Test the generate_response function with sample data and check token counting."""
    # Create a mock state
    test_state = {
        "question": "How do I treat a simple cut?",
        "retrived_docs": ["Clean the wound with mild soap and water. Apply antibiotic ointment and cover with a sterile bandage."],
        "summary": "This is the first question in the conversation."
    }
    
    # Call the function
    result = generate_response(test_state)
    
    # Print the results
    print("---TEST RESULTS---")
    print(f"Generated answer: {result['answer']}")
    print(f"Token usage: {result['token_usage']}")
    
    return result

# Run the test
test_result = test_generate_response()

---GENERATION OF RESPONSE---
---TEST RESULTS---
Generated answer: [AIMessage(content='To treat a simple cut, first clean the wound with mild soap and water. After cleaning, apply an antibiotic ointment to the cut and cover it with a sterile bandage.\n\n### Sources\n[1] Provided context', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--966b6533-e0cd-45a5-9534-f2ffdb3e3cf7-0')]
Token usage: [None]


# Web search

In [12]:
from tavily import TavilyClient

tavily_search = TavilyClient(api_key=TAVILY_API_KEY)

In [13]:
def web_search(state):
    '''
    Search the internet using, tavily

    Args:
        state(dict) : The current graph state.

    Return:
        Dict: New state with a key "retrived_docs"
    
    '''

    print("---WEB_SEARCH---")
    question = state['question']
    search_response = tavily_search.search(question) 
    if "results" not in search_response:
        raise ValueError("Invalid response from Tavily API")
     
    documents = [{"url": result['url'], "content": result['content']} for result in search_response['results']]

    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in documents
        ]
    )
    return {"retrived_docs" : formatted_search_docs}

# Route question

Based on the question the Router decides whether to direct the question to retrieve context from vectorstore or perform a web search.

In [14]:
from pydantic import BaseModel, Field
from typing import Literal
class RouteQuery(BaseModel):
    source : Literal["retrive", "web_search"] = Field(... , description="Given a user question route to web_search or to a retrive")

query_llm = model_for_query_route.with_structured_output(RouteQuery)

route_prompt = ChatPromptTemplate.from_messages([
    ("system", route_question_template),
    ("human", "{question}")
])

router = route_prompt | query_llm
print(router.invoke({"question": "Give me a  guidelines for minor scraps?"}))
print(router.invoke({"question": "Who is the current president of Frane?"}))

source='retrive'
source='web_search'


In [15]:
def route_question(state):
    """
    Route question to web_search or llm 
    Args:
        state (dict): The current graph state.

    Returns:
        dict: New node to call 
    """
    
    print("---ROUTING---")

    question = state['question']

    route = router.invoke({"question" : question})
    
    if route.source == "retrive":
        print("ROUTE QUESTION TO RAG")
        return "retrive"
        
    elif route.source == "web_search":
        print("ROUTE QUESTION TO WEB SEARCH")
        return "web_search"
    

# Memory

In [16]:
def summarize_conversation(state):
    """
    Updates the conversation summary by incorporating the latest question and answer.
    
    Args:
        state (dict): The current graph state

    Returns:
        dict: Updates the summary key with the extended or newly created summary
    """
    print("---SUMMARIZE_CONVERSATION---")
    
   
    question = state.get("question", "")
    answer = state.get("answer", "")
    token_count = state.get("token_usage", [])
    
    history = state.get("history", [])
    if not isinstance(history, list):
        history = []  

    new_exchange = {"question": question, "answer": answer}
    history.append(new_exchange)

    history_text = "\n".join([f"Q: {ex['question']}\nA: {ex['answer']}" for ex in history])
    summary_prompt = f"Current conversation summary:\n{history_text}\n\nExtend the summary with the new exchange:\nQ: {question}\nA: {answer}"
    
    response = model_for_query_route.invoke(summary_prompt)
    tokens = response.usage_metadata
    token_count.append(tokens)
    state["token_usage"] = token_count
    
    return {
        "history": history,
        "summary": response 
        
    }

# Corrective Retrieval Augmented Generation
https://arxiv.org/pdf/2401.15884

Corrective-RAG (CRAG) is a strategy for RAG that incorporates self-reflection / self-grading on retrieved documents.

With this feature, the system works more intelligently - it doesn't generate answers based on irrelevant or incomplete data, but instead automatically looks for better information on its own when necessary.
This improves the accuracy and quality of answers.

### Document grader

In [17]:
class GradeDocuments(BaseModel):
    score : bool = Field(... , description="Documents are relevant to the question, 'True' or 'False'")

grader_llm = model_for_query_route.with_structured_output(GradeDocuments)
grade_prompt = ChatPromptTemplate.from_messages([
    ("system" , grade_documents_template),
    ("human", "DOCUMENTS: \n\n {documents} \n\n QUESTION: {question}")
])

documents_grader = grade_prompt | grader_llm

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---GRADE RETRIVED DOCUMENTS---")
    question = state.get("question")
    retrived_docuemnts = state.get("retrived_docs")

    filtere_documents = []

    for doc in retrived_docuemnts:

        score = documents_grader.invoke({"documents" : doc , "question" : question})

        if score.score == True:
            filtere_documents.append(doc)
        else:
            continue

    score = documents_grader.invoke({"documents" : retrived_docuemnts , "question" : question})
    print(score.score)
    return {"retrived_docs" : filtere_documents,"are_documents_relevant" : score.score}
    

def route_graded(state):
    """ Route the research based on the documents relevance """
    
    score = state.get("are_documents_relevant")

    if score == True:
        print("ROUTE TO GENERATE")
        return "generate_response"
    else:
        print("ROUTE TO WEB SEARCH")
        return "web_search"
        

### Hallucination checker, Answer grader

In [18]:
class CheckHallucination(BaseModel):
    score : bool = Field(... , description="Answer are relevant to the retrived documents, True or False")
    
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    grade: bool = Field(
        description="Answer addresses the question, True or False"
    )
hallucination_checker_llm = model_for_query_route.with_structured_output(CheckHallucination)
structured_llm_grader = model_for_query_route.with_structured_output(GradeAnswer)



check_hallucination_prompt = ChatPromptTemplate.from_messages([
    ("system" , check_hallucination_template),
    ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {answer}"),
])

answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", answer_question_template),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
hallucination_checker = check_hallucination_prompt | hallucination_checker_llm

def check_hallucination_grade_answer(state):
    """
    Check hallucination and grade the answer. If the process fails twice, skip further checks and return the current answer.
    
    Args:
        state (dict): The current graph state.
    
    Returns:
        str: The next node to call.
    """
    print("---CHECK HALLUCINATION AND GRADE ANSWER---")

    question = state.get("question")
    documents = state.get("retrived_docs")
    answer = state.get("answer") 
    score = hallucination_checker.invoke({"documents" : documents , "answer" : answer})

    # Initialize or increment the retry counter
    retry_count = state.get("retry_count", 0)
    state["retry_count"] = retry_count + 1

    # If this is the third attempt, skip checks and return the current answer
    if retry_count >= 2:
        print("---MAX RETRIES REACHED: RETURNING CURRENT ANSWER---")
        return "summarize"
    # Check hallucination
    
    print("---GRADE GENERATION vs QUESTION---")
    grade = answer_grader.invoke({"question": question, "generation": answer})
    grade = grade.grade


    if grade == True:
        print("---DECISION: GENERATION ADDRESSES QUESTION---")
        return "summarize"
    else:
        print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
        return "not useful"

In [19]:
def test_hallucination_and_grade_answer():
    """
    Test the check_hallucination_grade_answer function, hallucination checker, and answer grader
    with various test cases.
    
    This tests:
    1. Hallucination detection (facts vs generation)
    2. Answer grading (question vs generation)
    3. Proper retry handling
    4. Edge cases
    """
    # Test Hallucination Checker
    print("=== TESTING HALLUCINATION CHECKER ===")
    hallucination_test_cases = [
        {
            "documents": "Honey bee stings should be removed by scraping with a flat edge.",
            "answer": "Remove a honey bee sting by scraping it off with a credit card or similar flat object.",
            "expected": True,
            "description": "Factually aligned generation"
        },
        {
            "documents": "Honey bee stings should be removed by scraping with a flat edge.",
            "answer": "You should always use tweezers to remove bee stings by pulling them out.",
            "expected": False,
            "description": "Contradictory generation"
        },
        {
            "documents": "Minor cuts should be cleaned with soap and water.",
            "answer": "Clean cuts with mild soap and water, then apply antibiotic ointment.",
            "expected": False,
            "description": "Generation with added information not in documents"
        }
    ]
    
    for i, test in enumerate(hallucination_test_cases):
        print(f"\nTest {i+1}: {test['description']}")
        result = hallucination_checker.invoke({
            "documents": test["documents"],
            "answer": test["answer"]
        })
        print(f"Expected: {test['expected']}, Got: {result.score}")
        print(f"PASS: {result.score == test['expected']}")
    
    # Test Answer Grader
    print("\n=== TESTING ANSWER GRADER ===")
    grader_test_cases = [
        {
            "question": "How do I treat a bee sting?",
            "generation": "To treat a bee sting, remove the stinger by scraping it off, wash with soap and water, apply ice to reduce swelling, and take antihistamines if needed.",
            "expected": True,
            "description": "Relevant and complete answer"
        },
        {
            "question": "How do I treat a bee sting?",
            "generation": "Bees are flying insects closely related to wasps and ants, and are known for their role in pollination.",
            "expected": False,
            "description": "Off-topic answer"
        },
        {
            "question": "What should I do for a deep cut that won't stop bleeding?",
            "generation": "For any cut, clean it with soap and water.",
            "expected": False,
            "description": "Incomplete answer for a serious condition"
        }
    ]
    
    for i, test in enumerate(grader_test_cases):
        print(f"\nTest {i+1}: {test['description']}")
        result = answer_grader.invoke({
            "question": test["question"],
            "generation": test["generation"]
        })
        print(f"Expected: {test['expected']}, Got: {result.grade}")
        print(f"PASS: {result.grade == test['expected']}")
    
    # Test the check_hallucination_grade_answer function with mock state
    print("\n=== TESTING CHECK_HALLUCINATION_GRADE_ANSWER FUNCTION ===")
    
    # Test Case 1: First attempt - good answer
    test_state_1 = {
        "question": "How do I treat a bee sting?",
        "retrived_docs": "Remove the stinger by scraping it with a flat edge. Clean with soap and water. Apply cold compress.",
        "answer": "To treat a bee sting, first remove the stinger by scraping it with a credit card or similar flat edge. Then wash the area with soap and water, and apply a cold compress to reduce swelling."
    }
    
    result_1 = check_hallucination_grade_answer(test_state_1)
    print(f"Test case 1 - First attempt with good answer")
    print(f"Result: {result_1}")
    print(f"Retry count: {test_state_1.get('retry_count', 0)}")
    
    # Test Case 2: First attempt - hallucination or off-topic
    test_state_2 = {
        "question": "How do I treat a bee sting?",
        "retrived_docs": "Remove the stinger by scraping it with a flat edge. Clean with soap and water. Apply cold compress.",
        "answer": "Bees are flying insects known for their role in pollination and honey production."
    }
    
    result_2 = check_hallucination_grade_answer(test_state_2)
    print(f"\nTest case 2 - First attempt with irrelevant answer")
    print(f"Result: {result_2}")
    print(f"Retry count: {test_state_2.get('retry_count', 0)}")
    
    # Test Case 3: Max retries reached
    test_state_3 = {
        "question": "How do I treat a bee sting?",
        "retrived_docs": "Remove the stinger by scraping it with a flat edge. Clean with soap and water. Apply cold compress.",
        "answer": "Bees are flying insects known for their role in pollination and honey production.",
        "retry_count": 2  # Already failed twice
    }
    
    result_3 = check_hallucination_grade_answer(test_state_3)
    print(f"\nTest case 3 - Max retries reached")
    print(f"Result: {result_3}")
    print(f"Retry count: {test_state_3.get('retry_count', 0)}")
    
    return "All tests completed"

# Execute the tests
if __name__ == "__main__":
    test_hallucination_and_grade_answer()

=== TESTING HALLUCINATION CHECKER ===

Test 1: Factually aligned generation
Expected: True, Got: False
PASS: False

Test 2: Contradictory generation
Expected: False, Got: False
PASS: True

Test 3: Generation with added information not in documents
Expected: False, Got: False
PASS: True

=== TESTING ANSWER GRADER ===

Test 1: Relevant and complete answer
Expected: True, Got: False
PASS: False

Test 2: Off-topic answer
Expected: False, Got: False
PASS: True

Test 3: Incomplete answer for a serious condition
Expected: False, Got: False
PASS: True

=== TESTING CHECK_HALLUCINATION_GRADE_ANSWER FUNCTION ===
---CHECK HALLUCINATION AND GRADE ANSWER---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---
Test case 1 - First attempt with good answer
Result: not useful
Retry count: 1
---CHECK HALLUCINATION AND GRADE ANSWER---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---

Test case 2 - First attempt with irrelevant answe

In [20]:
print(hallucination_checker.invoke({
    "documents": "The capital of France is Paris.",
    "answer": "Paris is the capital of France."
}))

print(hallucination_checker.invoke({
    "documents": "The capital of France is Paris.",
    "answer": "The capital of France is Lyon."
}))

score=True
score=False


In [21]:
print(answer_grader.invoke({
    "question": "Who discovered penicillin?",
    "generation": "Alexander Fleming discovered penicillin in 1928."
}))

print(answer_grader.invoke({
    "question": "What is the boiling point of water?",
    "generation": "Water is an important molecule for life."
}))

grade=True
grade=False


# Graph State

In [22]:
from langgraph.prebuilt import ToolNode ,tools_condition
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict , Annotated , List , Dict
from langchain_core.messages import AIMessage, AnyMessage, HumanMessage, SystemMessage, RemoveMessage
from langgraph.graph.message import add_messages
import operator

config = {"configurable" : {"thread_id": "1"}}
memory = MemorySaver()

class MedicalAssistantState(TypedDict):
    image: str
    image_metadata: dict
    question: str
    answer: str
    retrived_docs : List[str] 
    history: List[Dict[str, str]]  
    summary: str
    are_documents_relevant : bool
    retry_count : int
    token_usage : List[Dict[str, int]]

In [23]:
builder_2 = StateGraph(MedicalAssistantState)
builder_2.add_node("analyze_image", analyze_image_node,)
builder_2.add_node("generate_response", generate_response)
builder_2.add_node("retrive" , retrive)
builder_2.add_node("web_search" , web_search)
builder_2.add_node("summarize" , summarize_conversation)
builder_2.add_node("grade_documents" , grade_documents)

builder_2.add_edge(START , "analyze_image")
builder_2.add_conditional_edges("analyze_image" , route_question , {"web_search" : "web_search" , "retrive" :"retrive"})
builder_2.add_edge("retrive" , "grade_documents")
builder_2.add_conditional_edges("grade_documents" , route_graded , {"generate_response" : "generate_response" , "web_search" : "web_search"})
builder_2.add_edge("web_search" , "generate_response")
builder_2.add_conditional_edges("generate_response" , check_hallucination_grade_answer , {"summarize" : "summarize"  , "not useful" : "generate_response"})
builder_2.add_edge("summarize", END)

test_graph= builder_2.compile()

In [24]:
#display(Image(test_graph.get_graph().draw_mermaid_png()))

In [25]:
builder = StateGraph(MedicalAssistantState)

builder.add_node("analyze_image", analyze_image_node,)
builder.add_node("generate_response", generate_response)
builder.add_node("retrive" , retrive)
builder.add_node("web_search" , web_search)


builder.add_edge(START, "analyze_image")
builder.add_edge("analyze_image", "web_search")
builder.add_edge("analyze_image", "retrive")
builder.add_edge("web_search", "generate_response")
builder.add_edge("retrive", "generate_response")
builder.add_edge("generate_response", END)

medical_assistant_parallel  = builder.compile()

In [26]:
img_path = r"E:\Deep_Learning\Skin diseases\data_multiclass\Wound\Cut\cut (15).jpg"

response = test_graph.invoke({"question" : "What should I do when something bites my hand?"} , config=config)
print(response)

---IMAGE ANALYSIS---
No image provided to input
---ROUTING---
ROUTE QUESTION TO WEB SEARCH
---WEB_SEARCH---
---GENERATION OF RESPONSE---
---CHECK HALLUCINATION AND GRADE ANSWER---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
---SUMMARIZE_CONVERSATION---
{'question': 'What should I do when something bites my hand?', 'answer': [AIMessage(content="If a human bite breaks your skin, it's important to seek medical care from a doctor, ideally within 24 hours, due to the high risk of infection. A health professional may offer antibiotics, especially if the bite involves high-risk areas like the hands, feet, face, genitals, skin over joints, or areas with poor circulation, or if you have a medical condition that increases your risk of wound infection. If the wound doesn’t stop bleeding or the bite has removed significant tissue, seek help at an emergency room. Cleaning and bandaging the wound are frequent treatments for human bites.\n\n### Sources\n[1] https:

In [27]:
for m in response['answer']:
    print(m) 

content="If a human bite breaks your skin, it's important to seek medical care from a doctor, ideally within 24 hours, due to the high risk of infection. A health professional may offer antibiotics, especially if the bite involves high-risk areas like the hands, feet, face, genitals, skin over joints, or areas with poor circulation, or if you have a medical condition that increases your risk of wound infection. If the wound doesn’t stop bleeding or the bite has removed significant tissue, seek help at an emergency room. Cleaning and bandaging the wound are frequent treatments for human bites.\n\n### Sources\n[1] https://www.healthline.com/health/human-bites  \n[2] https://www.wikihow.com/Treat-a-Human-Bite  \n[3] https://patient.info/treatment-medication/human-bites" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--3d7b8bfd-1c30-4e0f-8d24-cbcee9

In [28]:
response_2 = test_graph.invoke({"question" : "I've cut through what to do?" , "image" : img_path} , config=config)

---IMAGE ANALYSIS---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
---ROUTING---
ROUTE QUESTION TO RAG
---RETRIVE---
Retrieval query: This question relates to a medical image classified as: Wound with probability 0.9996616840362549
---GRADE RETRIVED DOCUMENTS---
False
ROUTE TO WEB SEARCH
---WEB_SEARCH---
---GENERATION OF RESPONSE---
---CHECK HALLUCINATION AND GRADE ANSWER---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
---SUMMARIZE_CONVERSATION---


In [29]:
for m in response_2['answer']:
    print(m) 

content='If you cut yourself, the first thing you should do is apply pressure with a clean bandage or cloth, and keep that part of your body elevated above the heart, if possible. You may be tempted to lift the towel or gauze every few minutes to check on the cut, but avoid the urge, as this can reopen the cut and cause more bleeding. Cover the area with a clean, sterile bandage, change the bandage daily to keep the cut clean and dry, and wash your hands again. If blood spurts out of your cut, you have an abdominal or chest wound, or bleeding is so severe that it pools on the ground or soaks through your clothes, then call 9-1-1. If the cut is painful, you can take over-the-counter pain medication to reduce discomfort. Go to Urgent Care if the cut is gaping open, deep or jagged around the edges.\n\n### Sources\n[1] https://www.self.com/story/stop-bleeding-cut  \n[2] https://www.consumerreports.org/health/first-aid/how-to-treat-a-cut-a1032626330/  \n[3] https://healthywithpardee.com/how

In [30]:
response['summary'].content

'Unfortunately, there is no new exchange to add to the summary as the response provided does not contain any additional information or a new question. The same answer is repeated, with no new context or details.\n\nIf you would like, I can help you create a new summary based on the original conversation, or we can continue from here and add any new exchanges that are added to the conversation. Let me know how I can assist!'

In [31]:
response_2['summary'].content

"Here is the extended conversation summary:\n\nQ: I've cut through what to do?\nA: [AIMessage(content='If you cut yourself, the first thing you should do is apply pressure with a clean bandage or cloth, and keep that part of your body elevated above the heart, if possible. You may be tempted to lift the towel or gauze every few minutes to check on the cut, but avoid the urge, as this can reopen the cut and cause more bleeding. Cover the area with a clean, sterile bandage, change the bandage daily to keep the cut clean and dry, and wash your hands again. If blood spurts out of your cut, you have an abdominal or chest wound, or bleeding is so severe that it pools on the ground or soaks through your clothes, then call 9-1-1. If the cut is painful, you can take over-the-counter pain medication to reduce discomfort. Go to Urgent Care if the cut is gaping open, deep or jagged around the edges.\\n\\n### Sources\\n[1] https://www.self.com/story/stop-bleeding-cut  \\n[2] https://www.consumerrep

In [32]:
response_2['history']

[{'question': "I've cut through what to do?",
  'answer': [AIMessage(content='If you cut yourself, the first thing you should do is apply pressure with a clean bandage or cloth, and keep that part of your body elevated above the heart, if possible. You may be tempted to lift the towel or gauze every few minutes to check on the cut, but avoid the urge, as this can reopen the cut and cause more bleeding. Cover the area with a clean, sterile bandage, change the bandage daily to keep the cut clean and dry, and wash your hands again. If blood spurts out of your cut, you have an abdominal or chest wound, or bleeding is so severe that it pools on the ground or soaks through your clothes, then call 9-1-1. If the cut is painful, you can take over-the-counter pain medication to reduce discomfort. Go to Urgent Care if the cut is gaping open, deep or jagged around the edges.\n\n### Sources\n[1] https://www.self.com/story/stop-bleeding-cut  \n[2] https://www.consumerreports.org/health/first-aid/how

In [33]:
response_2['token_usage']

[None, {'input_tokens': 866, 'output_tokens': 422, 'total_tokens': 1288}]

In [34]:
response['token_usage']

[None, {'input_tokens': 596, 'output_tokens': 86, 'total_tokens': 682}]